In [2]:
import MDAnalysis as mda
import matplotlib.pyplot as plt
import math
import numpy as np

c:\Users\jeffs\Documents\Research_DrShen\MSD\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
graphite_water_psf = r'C:\Users\jeffs\Documents\Research_DrShen\MSD\msd_water_box\data_files\graphite_water.psf'
graphite_trajectory_dcd = r'C:\Users\jeffs\Documents\Research_DrShen\MSD\msd_water_box\data_files\output_per_40ps.dcd'
global_uni = mda.Universe(graphite_water_psf,graphite_trajectory_dcd)
global_uni.trajectory[0]
SPCE_uni = global_uni.select_atoms('resname SPCE')

c:\Users\jeffs\Documents\Research_DrShen\MSD\.venv\Lib\site-packages\MDAnalysis\coordinates\DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [4]:
frame_positions = {} # {frame_n:[[x_0, y_0, z_0], ...] , ... ], ... }
for frame in range(len(global_uni.trajectory)):
    global_uni.trajectory[frame]
    frame_positions[frame] = SPCE_uni.positions

z_maxs = []
for frame in frame_positions.keys():
    val_1 = []
    val_2 = []
    val_3 = []
    for atom in frame_positions[frame]:
        val_1.append(atom[0])
        val_2.append(atom[1])
        val_3.append(atom[2])
    z_maxs.append(max(val_3))

average_z_max = sum(z_maxs)/len(z_maxs)
partition_count = 4
z_partitions = []

for i in range(int(global_uni.dimensions[2] / 10)):
    z_ceiling = i * 10
    if average_z_max < z_ceiling:
        for i in range(partition_count):
            z_partitions.append(z_ceiling / 4 * (i + 1))
        break

In [5]:
# Function to calculate MSD
def calculate_msd(position_dict):
    x_vals = np.array(position_dict['x_vals'])
    y_vals = np.array(position_dict['y_vals'])
    z_vals = np.array(position_dict['z_vals'])
    
    # Number of time points
    num_points = len(x_vals)
    
    # Initialize MSD array
    msd = np.zeros(num_points)
    
    # Calculate displacements and MSD
    for dt in range(1, num_points):
        displacements = []
        for t in range(num_points - dt):
            dx = x_vals[t + dt] - x_vals[t]
            dy = y_vals[t + dt] - y_vals[t]
            dz = z_vals[t + dt] - z_vals[t]
            displacement_squared = dx**2 + dy**2 + dz**2
            displacements.append(displacement_squared)
        
        msd[dt] = np.mean(displacements)
    
    return msd


In [18]:

for atom_num in range(len(SPCE_uni)):

    atom_1_pos = {'x_vals':[],'y_vals':[],'z_vals':[]}
    for frame in range(len(global_uni.trajectory)):
        global_uni.trajectory[frame]
        SPCE_positions = SPCE_uni.positions
        atom_1_pos['x_vals'].append(SPCE_positions[0][0]) # SPCE_uni[atom_number][x or y or z]
        atom_1_pos['y_vals'].append(SPCE_positions[0][1])
        atom_1_pos['z_vals'].append(SPCE_positions[0][2])


    atom_1_part_movement = []
    atom_1_part = {}
    ticker = -1

    for i in range(len(atom_1_pos['z_vals'])): # Looping through each position point for atom 1 throughout the trajectory

        if atom_1_pos['z_vals'][i] < z_partitions[0]:
            current_part = 0
        elif atom_1_pos['z_vals'][i] < z_partitions[1]:
            current_part = 1
        elif atom_1_pos['z_vals'][i] < z_partitions[2]:
            current_part = 2 
        elif atom_1_pos['z_vals'][i] < z_partitions[3]:
            current_part = 3
        # Asking what partition does the atom currently fall in

        if len(atom_1_part_movement) == 0 or atom_1_part_movement[-1] != current_part:
            ticker += 1
            atom_1_part[ticker] = {'x_vals':[],'y_vals':[],'z_vals':[]}
        # Asking, are we NOT in the same partition as 1 frame ago
        # If  we are not in same partition, create data structure for atoms in new partition to be appended to
        # (Also just creates stucture if we werent in a partition last frame [ie just started])

        atom_1_part[ticker]['x_vals'].append(atom_1_pos['x_vals'][i])
        atom_1_part[ticker]['y_vals'].append(atom_1_pos['y_vals'][i])
        atom_1_part[ticker]['z_vals'].append(atom_1_pos['z_vals'][i])
        atom_1_part_movement.append(current_part)
        # With the correct structure in place, we append our x,y,z values to the correct location and append our current_part location

    msd_dict = {}
    msd_dict[atom_num] = {}
    for key in atom_1_part.keys():
        msd_dict[atom_num][key] = calculate_msd(atom_1_part[key])
    
    if atom_num == 100:
        print(atom_num)
        break

# 100 atoms runtime: 16.0s
# Removed MSD runtime ^: 14.6s -> 8.75% of runtime
# Removed 2nd Section & MSD ^: 14.4s -> 2nd Section is 0% of runtime -> 1st Section is 90% of runtime
# Theoretical runtime: len(SPCE_uni.atoms) / 100 * 16 /60 [min] -> 27.15min


    

100


In [9]:
print(len(SPCE_uni.atoms))

10182


In [13]:
print(10182/100*16.0/60)

27.151999999999997
